Select images to preserve to remove/minimize overlap; then copy subset images to a new directory to facilitate ML detection 
Will need to know in advance every n images required to remove forward overlap (eg, for 50% overlap, every other image)
Assumes a CSV with filename and flightline # already exists for the UAS images

In [1]:
#Creates a CSV file of independent image names (ie, no forward overlap)
import pandas as pd

# Load the CSV file into a pandas dataframe
input_file = ''  # CSV file that includes image names and transect numbers, ideally filtered by desired transects
df = pd.read_csv(input_file)

# Create a new dataframe with every other row from the original dataframe; can replace "2" with any n value of images to preserve
subset_df = df.iloc[::2].copy()

# Optional: Reset the index of the new dataframe
subset_df.reset_index(drop=True, inplace=True)

# Save the subset dataframe to a new CSV file
output_file = 'independent_imgs.csv'  # Replace with the desired filename
subset_df.to_csv(output_file, index=False)

                      filename   latitude   longitude land  flightline
0    EP-01-26106_0010_0127.JPG  36.573412 -104.589655  yes           4
1    EP-01-26106_0010_0129.JPG  36.573889 -104.589634  yes           4
2    EP-01-26106_0010_0131.JPG  36.574332 -104.589652  yes           4
3    EP-01-26106_0010_0133.JPG  36.574759 -104.589613  yes           4
4    EP-01-26106_0010_0135.JPG  36.575220 -104.589732  yes           4
..                         ...        ...         ...  ...         ...
195  EP-01-26106_0012_0320.JPG  36.574904 -104.575832  yes          29
196  EP-01-26106_0012_0322.JPG  36.574392 -104.575883  yes          29
197  EP-01-26106_0012_0324.JPG  36.573957 -104.575868  yes          29
198  EP-01-26106_0012_0326.JPG  36.573551 -104.575866  yes          29
199  EP-01-26106_0012_0328.JPG  36.573146 -104.575871  yes          29

[200 rows x 5 columns]


In [6]:
#Copy independent images to new directory to run SAHI detections

import os
import shutil

# Define the source directory containing the images
source_directory = ''  # Replace with the path to your images directory

# Define the destination directory where the images will be copied
destination_directory = ''  # Replace with the path to your output directory

# Loop through each row in the subset dataframe
for index, row in subset_df.iterrows():
    # Extract the filename and flightline from the dataframe
    filename = row['filename']
    flightline = row['flightline']

    # Create a new folder for the flightline if it doesn't exist
    flightline_directory = os.path.join(destination_directory, str(flightline))
    os.makedirs(flightline_directory, exist_ok=True)

    # Get the source path of the image
    source_image_path = os.path.join(source_directory, filename)

    # Get the destination path for the image
    destination_image_path = os.path.join(flightline_directory, filename)

    try:
        # Copy the image to the respective flightline folder
        shutil.copy(source_image_path, destination_image_path)
        print(f"Image {filename} copied to {flightline_directory}.")
    except FileNotFoundError:
        print(f"Image {filename} not found in {source_directory}.")

print("Image copying completed successfully.")


Image EP-01-26106_0010_0127.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0129.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0131.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0133.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0135.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0137.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0139.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0141.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0143.JPG not found in C:/Users/rowanconverse/Desktop/eBee/2017-12-15_lake13-3/img/.
Image EP-01-26106_0010_0145.JPG not f

In [4]:
#Makes a circular clip using the image height as the radius, goes from the center of the image (ie, to yield the detection radius)

from PIL import Image, ImageDraw
import os

def clip_circle(image_path, output_path):
    # Open the image
    img = Image.open(image_path)
    width, height = img.size

    # Calculate circle parameters
    center_x = width // 2
    center_y = height // 2
    radius = height // 2

    # Create a mask
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((center_x - radius, center_y - radius, center_x + radius, center_y + radius), fill=255)

    # Apply the mask to the image
    result = Image.new('RGBA', (width, height))
    result.paste(img, mask=mask)
    
    # Convert the image to RGB mode
    result_rgb = result.convert('RGB')

    # Save the clipped image
    result_rgb.save(output_path)

# Specify input and output directories
input_directory = 'C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\\18a03\\originals'
output_directory = 'C:\\Users\\rowanconverse\\Desktop\\PowerAnalysis\\BdA\output\\circles'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Loop through the images in the input directory
for filename in os.listdir(input_directory):
    if filename.lower().endswith('.jpg'):
        input_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, filename)
        clip_circle(input_path, output_path)